In [1]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [2]:
consumer_key = "xeIhWJkCnG2QIBKpEchgIwU8D"
consumer_secret = "wHHqXH83pve96QsFIXHwR5YvXjAEX4du7TnQcxNSEBduN8N0r2"
access_token = "1387475576911319041-1LaoCpINdmQJXXIVtMeGEWpqoacUmr"
access_token_secret = "Eny0985EzyYOkP29a7kBcxUfSgmFReZgr0nqTOWUY52wp"

In [3]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [4]:
query = "India will win Test Cricket Championship"

In [5]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x00000204E8E29070>, _json={'created_at': 'Tue Jun 22 05:33:47 +0000 2021', 'id': 1407210186339614721, 'id_str': '1407210186339614721', 'full_text': "Looking to India's approach against Australia and New Zealand's approach against England in the last test, both the teams will go for win. 6 Sessions 3 innings 28 wickets. This championship needs a clear winner. The purpose of reviving Test cricket won't be fulfilled otherwise.", 'truncated': False, 'display_text_range': [0, 278], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1357268516039061505, 'id_str': '1357268516039061505', 'name': 'CricAddicts

### Get the tweets and some Attributes

In [6]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [7]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [8]:
stream(data=["India will win Test Cricket Championship"] , file_name='india_tcc_tweets')

In [9]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,Looking to India's approach against Australia ...,CricAddicts,273,14,"Gujarat, India",False,2,0,2021-06-22 05:33:47
1,Ashwin vs Williamson in Tests- \n\nMatches: 6\...,Cricket.com,10332,19606,,False,2,0,2021-06-20 16:49:24
2,RT @BETDAQ: ▶️ India v New Zealand\n\n😍 Who wi...,Horse Racing,21364,40,everywhere,False,0,2,2021-06-19 12:58:06
3,RT @BETDAQ: ▶️ India v New Zealand\n\n😍 Who wi...,Horse Racing,21364,40,everywhere,False,0,2,2021-06-19 12:57:02
4,RT @BETDAQ: ▶️ India v New Zealand\n\n😍 Who wi...,Horse Racing,21364,40,everywhere,False,0,3,2021-06-19 12:56:41


In [10]:
df.tail()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
73,Who will win world test championship \n India ...,Aman kumar,40,0,"Bengaluru, India",False,0,0,2021-06-14 06:36:41
74,New Zealand will have the advantage of having ...,Jeetbuzz,258,60,,False,3,0,2021-06-14 05:08:37
75,RT @IExpressSports: New Zealand will have the ...,dhiman_saab,389018,315,india,False,0,3,2021-06-14 03:39:39
76,RT @IExpressSports: New Zealand will have the ...,The Indian Express,795791,3926387,India,True,0,3,2021-06-14 03:18:18
77,New Zealand will have the advantage of having ...,Express Sports,172891,84606,New Delhi,True,23,3,2021-06-14 03:18:11


In [11]:
from textblob import TextBlob

In [12]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [13]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [14]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [15]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,Looking to India's approach against Australia ...,CricAddicts,273,14,"Gujarat, India",False,2,0,2021-06-22 05:33:47,Looking to India s approach against Australia ...
1,Ashwin vs Williamson in Tests- \n\nMatches: 6\...,Cricket.com,10332,19606,,False,2,0,2021-06-20 16:49:24,Ashwin vs Williamson in Tests Matches 6 Dismis...
2,RT @BETDAQ: ▶️ India v New Zealand\n\n😍 Who wi...,Horse Racing,21364,40,everywhere,False,0,2,2021-06-19 12:58:06,RT India v New Zealand Who will win the WorldT...
3,RT @BETDAQ: ▶️ India v New Zealand\n\n😍 Who wi...,Horse Racing,21364,40,everywhere,False,0,2,2021-06-19 12:57:02,RT India v New Zealand Who will win the WorldT...
4,RT @BETDAQ: ▶️ India v New Zealand\n\n😍 Who wi...,Horse Racing,21364,40,everywhere,False,0,3,2021-06-19 12:56:41,RT India v New Zealand Who will win the WorldT...


In [16]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,Looking to India's approach against Australia ...,CricAddicts,273,14,"Gujarat, India",False,2,0,2021-06-22 05:33:47,Looking to India s approach against Australia ...,Positive
1,Ashwin vs Williamson in Tests- \n\nMatches: 6\...,Cricket.com,10332,19606,,False,2,0,2021-06-20 16:49:24,Ashwin vs Williamson in Tests Matches 6 Dismis...,Positive
2,RT @BETDAQ: ▶️ India v New Zealand\n\n😍 Who wi...,Horse Racing,21364,40,everywhere,False,0,2,2021-06-19 12:58:06,RT India v New Zealand Who will win the WorldT...,Positive
3,RT @BETDAQ: ▶️ India v New Zealand\n\n😍 Who wi...,Horse Racing,21364,40,everywhere,False,0,2,2021-06-19 12:57:02,RT India v New Zealand Who will win the WorldT...,Positive
4,RT @BETDAQ: ▶️ India v New Zealand\n\n😍 Who wi...,Horse Racing,21364,40,everywhere,False,0,3,2021-06-19 12:56:41,RT India v New Zealand Who will win the WorldT...,Positive


### Analysis of tweet

In [17]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 Ashwin vs Williamson in Tests- 

Matches: 6
Dismissals: 5
Average: 29.60

Who do you think will win this battle at… https://t.co/VByGLJ9kXV

Clean tweet:
 Ashwin vs Williamson in Tests Matches 6 Dismissals 5 Average 29 60 Who do you think will win this battle at

Sentiment of the tweet:
 Positive


In [18]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @RafazTweets: Watch the royal and finale encounter between India and new zealand. Who will win the first ever tile of world test champio…

Clean tweet:
 RT Watch the royal and finale encounter between India and new zealand Who will win the first ever tile of world test champio

Sentiment of the tweet:
 Positive


In [19]:
n = 50
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 Who will win the inaugural World Test Championship Final. 

India 🇮🇳 or New Zealand 🇳🇿?

We’ll have commentary from… https://t.co/8e8wt4o62S

Clean tweet:
 Who will win the inaugural World Test Championship Final India or New Zealand We ll have commentary from

Sentiment of the tweet:
 Positive


In [20]:
df[df.Sentiment == 'Positive'].shape[0]

75

In [21]:
df[df.Sentiment == 'Neutral'].shape[0]

2

In [22]:
df[df.Sentiment == 'Negative'].shape[0]

1